# Fit the sinusoidal amplitudes

First I have to create a PDF for fitting. We will do this in a seperate notebook. [seperate notebook](PDF.ipynb)

First load the fit classes.

In [ ]:
gROOT->ProcessLine(".x $HSCODE/hsfit/LoadFit.C+")

Now load and compile the PhiAsymmetry PDF class you created.

In [ ]:
Loader::Compile("PhiAsymmetry.cxx");

Use an instance of the FitManager class to organise the fit and give it an ouput directory. I use the pwd variable to keep everything in the local directory, while applying the full path for PROOF.

In [ ]:
FitManager RF;
TString pwd = TString(gSystem->Getenv("PWD"))+"/";
RF.SetUp().SetOutDir(pwd+"outObs/");

Now load the experimentally measured variables (or observables). I must use the name of the variables in the tree.

Note the slighlty different notation for the Category PolState. If any events had a PolState of 0 for example, then they would not be included in the fit.

In [ ]:
RF.SetUp().LoadVariable("Phi[-180,180]"); 
RF.SetUp().LoadVariable("Pol[0,1]"); 
RF.SetUp().LoadCategory("PolState[Polp=1,Polm=-1]");

If I want to use any other variable, or example to apply a cut, I must load it as an AuxVar. The limits given will be applied to the datset. Here we will not apply it, but it is left commented out to show what you could do.

In [ ]:
 //RF.SetUp().LoadAuxVar("M1[0,10]"); //Load Aux Var, limits used as cut 
// RF.SetUp().AddCut("M1>2"); //Additional cut based on vars or aux vars

IMPORTANT here we set the event ID variable. Each event in your tree should have a unique id with which to synchronise with weights. This is useful as it allows us to break the data up, perform seperate fits then combine the weights afterwards. The weights are not written into the input tree, rather they are stored seperately (as HS::Weights) and combined when required. This allows you to use various sets of weights when performing fits or regenerate the weights after some corrections etc.

In [ ]:
RF.SetUp().SetIDBranchName("fgID");

Create and load into the fit manager my PDF class. The class is callled PhiAsymmetry, this object is given the name SigAsym, which can be used to assign simulated data later. Here I explicitly declare my fit parameters A and B with there initial values and ranges. Phi, Pol and PolState all match the variables loaded above.

In [ ]:
RF.SetUp().FactoryPDF("PhiAsymmetry::SigAsym( Phi,Pol,PolState,A[0,-1,1],B[0,-1,1] )");
RF.SetUp().LoadSpeciesPDF("SigAsym",1);

Split the data into bins to perform seperate fits on.

Again I am not going to use this here, but it is straightforward to split the data here, but if I did the follwing line would create 4 sets of data with different Eg ranges and fits would be performed to each one. This will also be applied to any simulated data.

In [ ]:
 //RF.Bins().LoadBinVar("Eg",4,3,4);

Another option for making multiple fits is to bootstrap. Here I split the data into Nboot distinct sets and fit each one. This also work with the LoadBinVar option, once the data is split into bins it is then further split into bootstrap sets.

In [ ]:
//RF.Data().BootStrap(10); //Nboot = 10

Now I load my data, here I will use both data to fit and MC data for the integral calculation. Note in the latter case the string "SigAsym" matches the name given to the PDF object above. In principal I could fit multiple PDFs and give them each different MC data.

The strings are treename, filename and PDF name (simulation only).

In [ ]:
RF.LoadData("MyModel",pwd+"Data.root");
RF.LoadSimulated("MyModel",pwd+"MC.root","SigAsym");

Now I attach the weights from my sPlot fit. I want to use the signal weights which were given the name "Signal" in the sPlot notebook.

In [ ]:
RF.Data().LoadWeights("Signal",pwd+"sPlotFit/Tweights.root");

Now run the fits. I use the Process classes which allow me to choose between running directly here on a single core or multicore via PROOF-lite. It doesn't make sense to run with PROOF unless multiple splits have been defined with LoadBinVar or you are using Bootstrap, in which case you should relate the number of cores requested to the number of splits or bootstraps.

In [ ]:
Here::Go(&RF);
//OR run with PROOF-LITE on N=4 cores (you can change the 4)
// Proof::Go(&RF,4);

All plots and fit parameters will be saved into the output directory Results.root file. If LoadBinVar splits were applied then they will be in directories related to the bin name.